<a href="https://colab.research.google.com/github/KillMonga130/daily-ai-ml-projects/blob/main/NLP_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install spacy scikit-learn transformers gradio

import spacy
!python -m spacy download en_core_web_sm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 10.2 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.24.7
    Uninstalling huggingface-hub-0.24.7:
      Successfully uninstalled huggingface-hub-0.24.7
     ━

In [3]:
data = [
    ("Hello", "greeting"),
    ("Hi", "greeting"),
    ("Goodbye", "goodbye"),
    ("See you later", "goodbye"),
    ("What services do you offer?", "service_inquiry"),
    ("Tell me about your services.", "service_inquiry"),
    ("I want to book an appointment", "booking"),
    ("Schedule a meeting for me", "booking"),
    ("What are your hours?", "faq"),
    ("Where are you located?", "faq"),
]

texts, labels = zip(*data)


In [9]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline

X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

intent_model = make_pipeline(CountVectorizer(), MultinomialNB())
intent_model.fit(X_train, y_train)

print("Train Accuracy:", intent_model.score(X_test, y_test))

Train Accuracy: 0.0


In [10]:
def predict_intent(text):
  return intent_model.predict([text])[0]

print(predict_intent("I'd like to know your services."))

service_inquiry


In [11]:
nlp = spacy.load("en_core_web_sm")

def extract_entities(text):
  doc = nlp(text)
  entities = [(ent.text, ent.label_) for ent in doc.ents]
  return entities

print(extract_entities("Book an appointment for next Monday at 10 AM"))

[('next Monday', 'DATE'), ('10 AM', 'TIME')]


In [12]:
def chatbot_response(user_input):
  intent = predict_intent(user_input)

  entities = extract_entities(user_input)

  if intent == "grreting":
    return "Hello! How can I assist you today?"
  elif intent == "service_inquiry":
    return "We offer various services, including consultations and scheduling. What would you like to know more about?"
  elif intent == "booking":
    if entities:
      return f"Booking confirmed for {entities[0][0]}."
    return "Please provide a date and time for your booking."
  elif intent == "faq":
    return "Our office hours are from 9 AM to 5 PM, Monday to Friday, and we're located downtown."
  elif intent == "goodbye":
    return "Goodbye! Have a great day!"
  else:
    return "I'm not sure how to help with that. Could you please clarify?"

print(chatbot_response("Hello!"))
print(chatbot_response("Can I book an appointment for next Monday?"))
print(chatbot_response("What services do you provide?"))


I'm not sure how to help with that. Could you please clarify?
Booking confirmed for next Monday.
We offer various services, including consultations and scheduling. What would you like to know more about?


In [14]:
import gradio as gr


def gradio_chatbot(user_input):
  return chatbot_response(user_input)

gr.Interface(
    fn=gradio_chatbot,
    inputs="text",
    outputs="text",
    title="Customer Service Chatbot",
    description="Ask the chatbot about services, book an apppointment, or just say hello!"
).launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://581d374ec7802096ca.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
